<a href="https://colab.research.google.com/github/Furkan-Simsek/fakeNewsDetectionWithAI/blob/main/fakeNewsDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/252 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/758 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

In [2]:
from datasets import load_dataset

ds = load_dataset("mrm8488/fake-news")

fake_news.csv:   0%|          | 0.00/111M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/44898 [00:00<?, ? examples/s]

In [3]:
# split the dataset into train and test sets
train_ds = ds["train"].train_test_split(test_size=0.2, seed=42)["train"]
test_ds = ds["train"].train_test_split(test_size=0.2, seed=42)["test"]

In [16]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
)

In [17]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=test_ds
)

In [19]:
def tokenize_function(example):
	return tokenizer(example["text"], padding="max_length", truncation=True)

# Tokenize train and test datasets
train_ds_tokenized = train_ds.map(tokenize_function, batched=True)
test_ds_tokenized = test_ds.map(tokenize_function, batched=True)

# Update trainer to use tokenized datasets
trainer.train_dataset = train_ds_tokenized
trainer.eval_dataset = test_ds_tokenized



In [13]:
import torch
torch.cuda.is_available()

True

In [20]:
import torch

if torch.cuda.is_available():
    model.to("cuda")

trainer.train()

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Step,Training Loss
10,0.000300
20,0.024600
30,0.008100
40,0.000300
50,0.029400
60,0.023600
70,0.029200
80,0.001000
90,0.000700
100,0.000500


Step,Training Loss
10,0.000300
20,0.024600
30,0.008100
40,0.000300
50,0.029400
60,0.023600
70,0.029200
80,0.001000
90,0.000700
100,0.000500


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


KeyboardInterrupt: 

In [26]:
from transformers import BertForSequenceClassification, BertTokenizerFast

loaded_model = BertForSequenceClassification.from_pretrained("results/checkpoint-500")
#loaded_tokenizer = BertTokenizerFast.from_pretrained("results/checkpoint-500")

In [30]:
# test model
from transformers import pipeline

classifier = pipeline("text-classification", model=loaded_model, tokenizer=tokenizer)

text = test_ds[0]["text"]
result = classifier(text)
print(result)

Device set to use cuda:0


[{'label': 'positive', 'score': 0.9999021291732788}]


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [31]:
# Evaluate the model
evaluation_results = trainer.evaluate()
print(evaluation_results)

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


{'eval_loss': 0.002720752265304327}


In [41]:
def classify_text(text):
  """
  Classifies the sentiment of a given text using the loaded model and tokenizer.

  Args:
    text: The input text string to classify.

  Returns:
    The classification result from the pipeline.
  """
  if 'classifier' not in locals():
    print("Error: Classifier pipeline not initialized. Please run the cell to create the classifier.")
    return None
  result = classifier(text)
  return result

# Example usage:
# third_party_text = "This is a third party text to classify."
# classification_result = classify_text(third_party_text)
# print(classification_result)

In [34]:
text = """LONDON, UK – A new report from a group identifying itself as the Global Tech-Wellness Consortium has issued a stark warning to smartphone users worldwide. The report claims that a specific radio frequency, emitted by over 90 percent of modern smartphones, is directly interfering with the human brains ability to enter deep sleep.
The lead researcher, a person named Dr. Evelyn Reed, stated in a public broadcast that the frequency, dubbed P-14, disrupts melatonin production. Melatonin is the key hormone that regulates our sleep-wake cycles. Dr. Reed explained that even when the phone is not in use, this signal is active, creating a constant, low-level disturbance in our environment.
The consortiums research, released directly on their new website and not yet verified by mainstream scientific bodies, suggests a direct correlation between the rollout of this signal over the past five years and a global increase in reported cases of insomnia, chronic fatigue, and daytime grogginess.
Online health communities and influencers are already sharing this information widely. The hashtag #SignalSleepless is trending. One widely shared post urges everyone to switch their phones to airplane mode at night. It reads, The phone companies know about this but are hiding it to protect their profits. You must share this information. It could be the reason you are always tired. Protect your loved ones from this invisible danger.
"""

In [37]:
classification_result = classify_text(text)
print(classification_result)

Error: Classifier pipeline not initialized. Please run the cell to create the classifier.
None


In [39]:
from transformers import pipeline

classifier = pipeline("text-classification", model=loaded_model, tokenizer=tokenizer)

Device set to use cuda:0


In [42]:
text = """LONDON, UK – A new report from a group identifying itself as the Global Tech-Wellness Consortium has issued a stark warning to smartphone users worldwide. The report claims that a specific radio frequency, emitted by over 90 percent of modern smartphones, is directly interfering with the human brains ability to enter deep sleep.
The lead researcher, a person named Dr. Evelyn Reed, stated in a public broadcast that the frequency, dubbed P-14, disrupts melatonin production. Melatonin is the key hormone that regulates our sleep-wake cycles. Dr. Reed explained that even when the phone is not in use, this signal is active, creating a constant, low-level disturbance in our environment.
The consortiums research, released directly on their new website and not yet verified by mainstream scientific bodies, suggests a direct correlation between the rollout of this signal over the past five years and a global increase in reported cases of insomnia, chronic fatigue, and daytime grogginess.
Online health communities and influencers are already sharing this information widely. The hashtag #SignalSleepless is trending. One widely shared post urges everyone to switch their phones to airplane mode at night. It reads, The phone companies know about this but are hiding it to protect their profits. You must share this information. It could be the reason you are always tired. Protect your loved ones from this invisible danger.
"""

classification_result = classify_text(text)
print(classification_result)

Error: Classifier pipeline not initialized. Please run the cell to create the classifier.
None


In [44]:
from transformers import pipeline

# Initialize the classifier pipeline
# Make sure loaded_model and loaded_tokenizer are available from previous cells
if 'loaded_model' not in locals() or 'tokenizer' not in locals():
    print("Error: loaded_model or loaded_tokenizer not found. Please ensure previous cells were run.")
else:
    classifier = pipeline("text-classification", model=loaded_model, tokenizer=tokenizer)

    # Text to classify
    text_to_classify = """LONDON, UK – A new report from a group identifying itself as the Global Tech-Wellness Consortium has issued a stark warning to smartphone users worldwide. The report claims that a specific radio frequency, emitted by over 90 percent of modern smartphones, is directly interfering with the human brains ability to enter deep sleep.
    The lead researcher, a person named Dr. Evelyn Reed, stated in a public broadcast that the frequency, dubbed P-14, disrupts melatonin production. Melatonin is the key hormone that regulates our sleep-wake cycles. Dr. Reed explained that even when the phone is not in use, this signal is active, creating a constant, low-level disturbance in our environment.
    The consortiums research, released directly on their new website and not yet verified by mainstream scientific bodies, suggests a direct correlation between the rollout of this signal over the past five years and a global increase in reported cases of insomnia, chronic fatigue, and daytime grogginess.
    Online health communities and influencers are already sharing this information widely. The hashtag #SignalSleepless is trending. One widely shared post urges everyone to switch their phones to airplane mode at night. It reads, The phone companies know about this but are hiding it to protect their profits. You must share this information. It could be the reason you are always tired. Protect your loved ones from this invisible danger.
    """

    # Classify the text
    classification_result = classifier(text_to_classify)
    print(classification_result)

Device set to use cuda:0


[{'label': 'positive', 'score': 0.9960287809371948}]


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [45]:
# Export the model and tokenizer
output_dir = "./exported_model"
loaded_model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)
print(f"Model and tokenizer exported to {output_dir}")

Model and tokenizer exported to ./exported_model


In [46]:
!zip -r exported_model.zip exported_model/

  adding: exported_model/ (stored 0%)
  adding: exported_model/config.json (deflated 51%)
  adding: exported_model/special_tokens_map.json (deflated 42%)
  adding: exported_model/model.safetensors (deflated 7%)
  adding: exported_model/vocab.txt (deflated 53%)
  adding: exported_model/tokenizer_config.json (deflated 75%)
  adding: exported_model/tokenizer.json (deflated 71%)


In [47]:
import torch

# Define the output path for the PyTorch model file
output_path = "./exported_model/pytorch_model.pt"

# Save the model's state dictionary
torch.save(loaded_model.state_dict(), output_path)

print(f"Model state dictionary exported to {output_path}")

Model state dictionary exported to ./exported_model/pytorch_model.pt
